In [13]:
import json
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import MobileApplicationClient

In [14]:
client_id = 'pt_UA7cYdAIc'
client_secret = 'p5k44_pSqh5dzK7MSC3MCDEmz81gnjXV'
scope = ['public']
url = 'https://api.lyft.com/oauth/token'

In [25]:
url = 'https://api.lyft.com/oauth/token'

# define request parameters
payload = {"Content-Type": "application/json",
           "grant_type": "client_credentials",
           "scope": "public"}
# request data
res = requests.post(url,
                    data = payload,
                    auth = (client_id, client_secret))
# extract the token from the response
token = res.json()['access_token']

In [41]:
query_url = 'https://api.lyft.com/v1/eta?'
header = {"Authorization": "Bearer {}".format(token)}
lat, long, rtype = 37.7833, -122.4167, 'lyft'
query = query_url + 'lat={:.4f}&lng={:.4f}&ride_type={}'.format(lat,long,rtype)
r = requests.get(query, headers=header)

In [43]:
r.json()

{'eta_estimates': [{'display_name': 'Lyft',
   'eta_seconds': 120,
   'is_valid_estimate': True,
   'ride_type': 'lyft'}],
 'timezone': 'America/Los_Angeles'}

In [44]:
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'